# WeRateDogs Twitter Feed

This project looks at various data sources for Tweets from the [WeRateDogs](https://twitter.com/dog_rates) Twitter account, specifically:

1. the `twitter-archive-enhanced.csv` which contains the tweet text and various related identifiers
1. the Twitter API is used to access the original tweets to retrieve missing fields such as the retweet and favorite counts
1. an image prediction file containing the top 3 predictions for each of the (up to 4) dog pictures in the tweet


The `twitter-archive-enhanced.csv` is the master source for the set of tweets to be included in this project. However, this dataset will need to be cleaned, and will then be enriched with additional attributes obtained through the [Twitter API](https://developer.twitter.com/en/docs/twitter-api), and finally enriched with data from the breed predictions file, which includes the top 3 predictions based on the dog's images. The data will be mapped into Pandas DataFrames folowing the Tidy Data principles.

---

## Gather data

The data assets used are listed below. Some are local files, downloaded into the `data` directory, some are remote files on web servers, and some are sourced via an API. Where there is enrichment of existing data, I will add the new attributes to the base DataFrame. Standalone data such as the breed predictions I will load into a separate DataFrame, so as to later reconcile back to the tweets.

In [1]:
!ls ./data

image-predictions.tsv        tweet-json.zip
tweet-json copy              twitter-archive-enhanced.csv


In [2]:
WE_RATE_DOGS_TWEETS_PATH = 'data/twitter-archive-enhanced.csv'
DOG_BREED_PREDICTIONS_SOURCE_URL = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

It is worth noting that the Pandas `read_csv()` function is quite versatile when uploading data, and can be configured to handle different date formats, numeric data types, not available (NA) value markers, etc. But to benefit from such functionality we need to eyeball the raw data first.

Let's look at the first 8 records, and 8 records from near the end of the file:-

In [3]:
!head -n 11 {WE_RATE_DOGS_TWEETS_PATH}

tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
892420643555336193,,,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,,,,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
892177421306343426,,,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",,,,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
891815181378084864,,,2017-

In [4]:
!tail -n 1940 {WE_RATE_DOGS_TWEETS_PATH} | head -n 8

813142292504645637,,,2016-12-25 22:00:04 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Everybody stop what you're doing and look at this dog with her tiny Santa hat. 13/10 https://t.co/KK4XQK9SPi,,,,"https://twitter.com/dog_rates/status/813142292504645637/photo/1,https://twitter.com/dog_rates/status/813142292504645637/photo/1,https://twitter.com/dog_rates/status/813142292504645637/photo/1",13,10,None,None,None,None,None
813130366689148928,8.13127251579564e+17,4196983835.0,2016-12-25 21:12:41 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I've been informed by multiple sources that this is actually a dog elf who's tired from helping Santa all night. Pupgraded to 12/10,,,,,12,10,None,None,None,None,None
813127251579564032,,,2016-12-25 21:00:18 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here's an anonymous doggo that appears to be very done w

A few observations:

1. the 1st line lists out the colummn names
1. tweet Ids are very large integres
1. some tweet Ids appear as floating point numbers, e.g.: in_reply_to_status_id, in_reply_to_user_id, as many are NaNs (i.e: nulls or missing values)
1. time stamps are almost in ISO 8601 (they are missing the 'T' separator) and are GMT (+00:00 offset) 
1. strings are wrapped in double quotes (") and embeded quotes appear as two contiguous quotes ("")
1. some strings such as the tweets with links to GoFundMe page have 2 embeded new lines within the double quotes
1. dog names and stages (last 5 columns) are extracted where found, otherwise the value _None_ is placed in those columns
1. whilst the dog name is variable, the stage is predefined, so putting the stage name into a column named after the stage name is redundant information
1. the 'source' column for all rows is an HTML anchor with a link to http://twitter.com/download/iphone which has no value, we can drop this column


Actions taken to address above observations:

* convert floating point tweets Ids to a 64-bit integer, retaining the Not Available representation
* specifcally tell Pandas which columns are dates
* drop the 'source' column

In [5]:
import yaml
import tweepy
import json

In [6]:
import numpy as np
import pandas as pd

Load the enhanced Twitter archive, using explicit data types for fields, instead of letting Pandas infer them. The [Twitter API](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview/tweet-object) will define the data types for the Twitter fields, to which I add the data types for the "enhanced" fields.

To get around the fact that nullable numeric fields, by default, are interpreted by `read_csv()` as floats (so as to include NaN to represent null or Not Available), I am mapping optional tweet Ids to Pandas nullable integer data type.

In [7]:
feed_data_types = {
    'tweet_id': np.int64,
    'in_reply_to_status_id': 'Int64',
    'in_reply_to_user_id': 'Int64',
    'retweeted_status_id': 'Int64',
    'retweeted_status_user_id': 'Int64',
    'text': 'string',
    'expanded_urls': 'string',
    'rating_numerator': np.int32,
    'rating_denominator': np.int32,
    'name': 'string',
    'doggo': 'string',
    'floofer': 'string',
    'pupper': 'string',
    'puppo': 'string'
}

feed_date_cols = [
    'timestamp', 
    'retweeted_status_timestamp'
]

In [8]:
tweets_df = pd.read_csv(WE_RATE_DOGS_TWEETS_PATH,
                        index_col=['tweet_id'],
                        dtype=feed_data_types,
                        parse_dates=feed_date_cols)
tweets_df.shape

(2356, 16)

The first discrepancy we note is that, according to the project motivation document, the main "archive contains basic tweet data for all 5000+ of their tweets" however that is clearly not the case as, having loaded it, the number of tweets is less than half that. As this is the master dataset we have been provided with, this is the data we will go with, since it has been previously enhanced.

Just to double check this row count, we will run a line count on the input file, which should roughly match the number of rows in the data frame. Any discrepancy on counts is due to embeded new line (NL) characters in tweet text, as has been previously mentioned.

In [9]:
!wc -l {WE_RATE_DOGS_TWEETS_PATH}

    2518 data/twitter-archive-enhanced.csv


In [10]:
tweets_df = tweets_df.drop(columns=['source'])
tweets_df.shape

(2356, 15)

In [11]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

Now we can doublecheck the column data types.

In [12]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 892420643555336193 to 666020888022790149
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   in_reply_to_status_id       78 non-null     Int64              
 1   in_reply_to_user_id         78 non-null     Int64              
 2   timestamp                   2356 non-null   datetime64[ns, UTC]
 3   text                        2356 non-null   string             
 4   retweeted_status_id         181 non-null    Int64              
 5   retweeted_status_user_id    181 non-null    Int64              
 6   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 7   expanded_urls               2297 non-null   string             
 8   rating_numerator            2356 non-null   int32              
 9   rating_denominator          2356 non-null   int32              
 10  name                        2

Get Twitter credentials and authenticate:-

In [13]:
def read_creds(conf_path):
    with open(conf_path, 'r') as cf:
        config = yaml.load(cf, Loader=yaml.FullLoader)
        return config

In [14]:
creds = read_creds('./config/private/creds.yaml')

In [15]:
consumer_key = creds['consumer_api']['key']
consumer_secret = creds['consumer_api']['secret']

In [16]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [17]:
access_token = creds['access_token']['token']
acess_secret = creds['access_token']['secret']

In [18]:
auth.set_access_token(access_token, acess_secret)

Setup the Twitter API and load the enrichment data in batches, for better performance, as API invocations are subject to significant network latency. Twitter applies rate limiting to their APIs, so it is necessary to throttle the rate at which we make requests, and to retry any failed API invocations. Tgis can be handled automatically by the Tweepy library, by setting the `wait_on_rate_limit_notify` flag when setting up the API connection.

Although we are enriching the core enhanced tweets archive, we will initially load the API data into a separate data frame, cleanup as necessary, and then merge from there into the main table.

In [19]:
api = tweepy.API(auth, wait_on_rate_limit_notify=True)

In [20]:
batch_size = 100
num_tweets = len(tweets_df.index)

In [21]:
def process_batch(batch):
    idxs = []
    retweet_counts = []
    favorite_counts = []
    for status in batch:
        tweet = status._json
        idxs.append(tweet['id'])
        retweet_counts.append(tweet['retweet_count'])
        favorite_counts.append(tweet['favorite_count'])
    return np.array(idxs, dtype=np.int64), np.array([retweet_counts, favorite_counts], dtype=np.int64).T

In [22]:
indices = np.empty((0), dtype=np.int64)
rows = np.empty((0, 2), dtype=np.int64)

In [23]:
for batch_start in range(0, num_tweets, batch_size):
    batch_end = min(batch_start + batch_size, num_tweets)
    batch_tweet_ids = tweets_df.iloc[batch_start:batch_end].index.to_numpy().tolist()
    statuses = api.statuses_lookup(batch_tweet_ids, include_entities=False, map_=False)
    b_indices, b_rows = process_batch(statuses)
    indices = np.concatenate((indices, b_indices), axis=0)
    rows = np.concatenate((rows, b_rows), axis=0)

In [24]:
tweet_counts_df = pd.DataFrame(index=indices, data=rows, 
                               columns=['retweet_counts', 'favorite_counts'], 
                               dtype='Int32')
tweet_counts_df.index.name = 'tweet_id'

In [25]:
enriched_tweets_df = tweets_df.merge(tweet_counts_df, how='left', on='tweet_id')
enriched_tweets_df.shape

(2356, 17)

Now lets review the structure of the merged data frame.

In [26]:
enriched_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 892420643555336193 to 666020888022790149
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   in_reply_to_status_id       78 non-null     Int64              
 1   in_reply_to_user_id         78 non-null     Int64              
 2   timestamp                   2356 non-null   datetime64[ns, UTC]
 3   text                        2356 non-null   string             
 4   retweeted_status_id         181 non-null    Int64              
 5   retweeted_status_user_id    181 non-null    Int64              
 6   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 7   expanded_urls               2297 non-null   string             
 8   rating_numerator            2356 non-null   int32              
 9   rating_denominator          2356 non-null   int32              
 10  name                        2

Finally we need to load the image predictions data, so we can later tidy it. We will read this data from the CloudFront URL, a opposed to the local file, to ensure we get the most up-to-date version. Note the suffix of this file (.TSV) which indicates it is tab character (not comma) separated. Once loaded we can review the data frame structure.

In [27]:
img_preds_data_types = {
    'tweet_id': 'Int64',
    'jpg_url': 'string',
    'img_num': np.int32,
    'p1': 'string',
    'p1_conf': np.float32,
    'p1_dog': bool,
    'p2': 'string',
    'p2_conf': np.float32,
    'p2_dog': bool,
    'p3': 'string',
    'p3_conf': np.float32,
    'p3_dog': bool
}

In [28]:
img_preds_df = pd.read_csv(DOG_BREED_PREDICTIONS_SOURCE_URL,
                           index_col=['tweet_id'],
                           sep='\t', 
                           dtype=img_preds_data_types)
img_preds_df.shape

(2075, 11)

In [29]:
img_preds_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2075 entries, 666020888022790149 to 892420643555336193
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   jpg_url  2075 non-null   string 
 1   img_num  2075 non-null   int32  
 2   p1       2075 non-null   string 
 3   p1_conf  2075 non-null   float32
 4   p1_dog   2075 non-null   bool   
 5   p2       2075 non-null   string 
 6   p2_conf  2075 non-null   float32
 7   p2_dog   2075 non-null   bool   
 8   p3       2075 non-null   string 
 9   p3_conf  2075 non-null   float32
 10  p3_dog   2075 non-null   bool   
dtypes: bool(3), float32(3), int32(1), string(4)
memory usage: 119.6+ KB


---

## Assess data

We have already assessed much of the enhanced tweets data, and fixed a few issues at load time. From a structural perspective we need to look at redundant data, specificly the dog stage columns, each of which can be converted to a binary flag. 

Also missing names appear to use the name "None" as opposed to a Pandas NA marker, as do the dog stage columns.

Let's take a quick look at the data first.

In [31]:
display(enriched_tweets_df.head(10))

,in_reply_to_status_id,in_reply_to_user_id,timestamp,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,retweet_counts,favorite_counts
tweet_id,,,,,,,,,,,,,,,,,
892420643555336193,<NA>,<NA>,2017-08-01 16:23:56+00:00,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,<NA>,<NA>,NaT,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None,7499,35505
892177421306343426,<NA>,<NA>,2017-08-01 00:17:27+00:00,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",<NA>,<NA>,NaT,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None,5565,30732
891815181378084864,<NA>,<NA>,2017-07-31 00:18:03+00:00,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,<NA>,<NA>,NaT,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None,3685,23111
891689557279858688,<NA>,<NA>,2017-07-30 15:58:51+00:00,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,<NA>,<NA>,NaT,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None,7679,38796
891327558926688256,<NA>,<NA>,2017-07-29 16:00:24+00:00,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",<NA>,<NA>,NaT,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None,8286,37067
891087950875897856,<NA>,<NA>,2017-07-29 00:08:17+00:00,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,<NA>,<NA>,NaT,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,None,None,None,None,None,2774,18691
890971913173991426,<NA>,<NA>,2017-07-28 16:27:12+00:00,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below https://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,<NA>,<NA>,NaT,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",13,10,Jax,None,None,None,None,1801,10862
890729181411237888,<NA>,<NA>,2017-07-28 00:22:40+00:00,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,<NA>,<NA>,NaT,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,10,None,None,None,None,None,16800,59852
890609185150312448,<NA>,<NA>,2017-07-27 16:25:51+00:00,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,<NA>,<NA>,NaT,https://twitter.com/dog_rates/status/890609185150312448/photo/1,13,10,Zoey,None,None,None,None,3825,25709


So now we convert those dog stage columns into boolean data type.

In [33]:
stage_cols = ['doggo', 'floofer', 'pupper', 'puppo']

In [34]:
enriched_tweets_df[stage_cols] = enriched_tweets_df[stage_cols].apply(lambda c: c.to_numpy() != 'None')

In [35]:
enriched_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 892420643555336193 to 666020888022790149
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   in_reply_to_status_id       78 non-null     Int64              
 1   in_reply_to_user_id         78 non-null     Int64              
 2   timestamp                   2356 non-null   datetime64[ns, UTC]
 3   text                        2356 non-null   string             
 4   retweeted_status_id         181 non-null    Int64              
 5   retweeted_status_user_id    181 non-null    Int64              
 6   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 7   expanded_urls               2297 non-null   string             
 8   rating_numerator            2356 non-null   int32              
 9   rating_denominator          2356 non-null   int32              
 10  name                        2

The other obvious issue is with the `expanded_urls` column, which stores the full length URL for shortened URLs that appear in the tweet:

1. it can store multiple URLs per row, as a comma separated string
1. the same URL can appear multiple times

It's not obvious looking at the tweet text, and at the number of times a link appears in the text, why the same URL sometimes gets repeated. So instead of storing the repetition count in another column, we can just clean out any duplicates. The first step is to normalise the data in a new data frame, where we can do te cleaning.

In [36]:
# Pull out rows containing one or more expanded URLs

expanded_urls_ser = enriched_tweets_df.loc[enriched_tweets_df['expanded_urls'].isna() == False]['expanded_urls']

In [37]:
# Nested list comprehension to split multiple URL strings on comma separator, then create [tweet Id, URL] tuples

expanded_url_tuples = [(ix, url) for ix, urls in expanded_urls_ser.iteritems() for url in urls.split(',')]

In [38]:
expanded_url_df = pd.DataFrame(expanded_url_tuples, columns=['tweet_id', 'expanded_url'])

In [39]:
# Now drop duplicates and make 'tweet_id' the index for consistency with other data frames

expanded_url_df = expanded_url_df.drop_duplicates().set_index('tweet_id')

In [40]:
expanded_url_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2338 entries, 892420643555336193 to 666020888022790149
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   expanded_url  2338 non-null   object
dtypes: object(1)
memory usage: 36.5+ KB


Finally we will tidy the image predictions data, and remove variables (the prediction number) from the column name.

In [44]:
def slice_pred_cols(df, numeric):
    preds_df = df[['jpg_url', 'img_num']]
    preds_df = preds_df.assign(pred_rank=numeric,
                               pred_class=img_preds_df[f'p{numeric}'],
                               pred_confidence=img_preds_df[f'p{numeric}_conf'],
                               pred_is_dog=img_preds_df[f'p{numeric}_dog'])
    return preds_df

In [45]:
len(img_preds_df.index)

2075

In [46]:
preds1_df = slice_pred_cols(img_preds_df, 1)
preds2_df = slice_pred_cols(img_preds_df, 2)
preds3_df = slice_pred_cols(img_preds_df, 3)
predictions_df = pd.concat([preds1_df, preds2_df, preds3_df]).sort_values(by=['tweet_id', 'pred_rank'])

In [47]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6225 entries, 666020888022790149 to 892420643555336193
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   jpg_url          6225 non-null   string 
 1   img_num          6225 non-null   int32  
 2   pred_rank        6225 non-null   int64  
 3   pred_class       6225 non-null   string 
 4   pred_confidence  6225 non-null   float32
 5   pred_is_dog      6225 non-null   bool   
dtypes: bool(1), float32(1), int32(1), int64(1), string(2)
memory usage: 249.2 KB


---

## Clean data

We have already performed a few cleaning tasks, such as sorting out data types and redundant values.

In the following sections we will clean up the data as specified under Key Points, in the Project Motivation page.

In [49]:
# Delete retweets, then drop the columns related to retweets as the remaining rows will all be NA

enriched_tweets_df = enriched_tweets_df.loc[enriched_tweets_df['retweeted_status_timestamp'].isna()]
enriched_tweets_df = enriched_tweets_df.drop(columns=['retweeted_status_id', 
                                                      'retweeted_status_user_id', 
                                                      'retweeted_status_timestamp'])
enriched_tweets_df.shape

(2175, 14)

In [50]:
# Delete tweets without images

enriched_tweets_df = enriched_tweets_df.loc[enriched_tweets_df.index.intersection(img_preds_df.index, sort=None)]
enriched_tweets_df.shape

(1994, 14)

As there appear to be no tweets beyond August 1st, 2017 (most likely since we dropped tweets witjout images) we are now done with cleaning.